# 自动微分

`Ascend` `GPU` `CPU` `入门` `模型开发`

[![在线运行](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_modelarts.png)](https://authoring-modelarts-cnnorth4.huaweicloud.com/console/lab?share-url-b64=aHR0cHM6Ly9taW5kc3BvcmUtd2Vic2l0ZS5vYnMuY24tbm9ydGgtNC5teWh1YXdlaWNsb3VkLmNvbS9ub3RlYm9vay9tb2RlbGFydHMvcXVpY2tfc3RhcnQvbWluZHNwb3JlX2F1dG9ncmFkLmlweW5i&imageid=65f636a0-56cf-49df-b941-7d2a07ba8c8c)&emsp;[![下载Notebook](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/mindspore_autograd.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/mindspore_autograd.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/beginner/autograd.ipynb)

自动微分是网络训练中常用的反向传播算法的一般化，利用该算法用户可以将多层复合函数分解为一系列简单的基本运算，该功能让用户可以跳过复杂的求导过程的编程，从而大大降低框架的使用门槛。
MindSpore使用`ops.GradOperation`计算一阶导数，`ops.GradOperation`属性如下：

+ `get_all`：对输入求导，为`True`时，对所有输入求导，为`False`时，只会对第一个输入求导，默认值为`False`。
+ `get_by_list`：对权重求导，为`True`时，对权重求导，为`False`时，不对权重求导，默认值为`False`。
+ `sens_param`：对网络的输出值做缩放以改变最终梯度，为`True`时，进行缩放操作，为`False`时，不进行缩放操作，默认值为`False`。

本章使用MindSpore中的`ops.GradOperation`对函数 $y=wx+b$ 求一阶导数。

## 对输入求一阶导

对输入求导前需要先定义$y=wx+b$网络。

In [109]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor, Parameter


class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(Tensor(np.array([6], np.float32)), name='w')
        self.b = Parameter(Tensor(np.array([1.0], np.float32)), name='b')

    def construct(self, x):
        out = x * self.w + self.b
        return out

然后定义求导类`GradNetWrtX`，类的`__init__`函数中定义需要求导的网络`self.net`和`ops.GradOperation`操作，类的`construct`函数中对`self.net`进行求导。

In [103]:
class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()

    def construct(self, x):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x)

最后定义输入，并对输入求一阶导数。

In [104]:
x = Tensor([6], dtype=mstype.float32)
output = GradNetWrtX(Net())(x)
print(output)

[6.]


[WARNING] OPTIMIZER(19940,7f1b6134b740,python):2022-02-12-01:41:49.171.536 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1634_construct_wrapper.411, J user: 1634_construct_wrapper.411:construct{[0]: 412, [1]: x, [2]: ValueNode<UMonad> U}


## 对权重求一阶导

对权重参数求一阶导，需要将`ops.GradOperation`中的`get_by_list`设置为`True`。

In [105]:
class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)

# 构建求导网络
x = Tensor([6], dtype=mstype.float32)
output = GradNetWrtX(Net())(x)
print(output)
print(f"wgrad: {output[0]}\nbgrad: {output[1]}")

(Tensor(shape=[1], dtype=Float32, value= [ 6.00000000e+00]), Tensor(shape=[1], dtype=Float32, value= [ 1.00000000e+00]))
wgrad: [6.]
bgrad: [1.]


[WARNING] OPTIMIZER(19940,7f1b6134b740,python):2022-02-12-01:41:49.998.167 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1674_construct_wrapper.419, J user: 1674_construct_wrapper.419:construct{[0]: 420, [1]: x, [2]: ValueNode<UMonad> U}


若某些权重不需要进行求导，则在定义求导网络时，相应的权重参数属性`requires_grad`需设置为`False`。

In [106]:
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(Tensor(np.array([6], np.float32)), name='w')
        self.b = Parameter(Tensor(np.array([1.0], np.float32)), name='b', requires_grad=False)

    def construct(self, x):
        out = x * self.w + self.b
        return out


class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)


# 构建求导网络
x = Tensor([6], dtype=mstype.float32)
output = GradNetWrtX(Net())(x)
print(output)

(Tensor(shape=[1], dtype=Float32, value= [ 6.00000000e+00]),)


[WARNING] OPTIMIZER(19940,7f1b6134b740,python):2022-02-12-01:41:50.956.836 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1715_construct_wrapper.426, J user: 1715_construct_wrapper.426:construct{[0]: 427, [1]: x, [2]: ValueNode<UMonad> U}


## 梯度值缩放

通过`sens_param`参数对网络的输出值做缩放以改变最终梯度。首先将`ops.GradOperation`中的`sens_param`设置为`True`，并确定缩放指数，其维度与输出维度保持一致。

In [107]:
class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        # 求导操作
        self.grad_op = ops.GradOperation(sens_param=True)
        # 缩放指数
        self.grad_wrt_output = Tensor([0.1], dtype=mstype.float32)

    def construct(self, x):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, self.grad_wrt_output)


x = Tensor([6], dtype=mstype.float32)
output = GradNetWrtX(Net())(x)
print(output)

[0.6]


[WARNING] OPTIMIZER(19940,7f1b6134b740,python):2022-02-12-01:41:51.837.649 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1756_construct_wrapper.433, J user: 1756_construct_wrapper.433:construct{[0]: [CNode]434, [1]: x, [2]: ValueNode<UMonad> U}


## 停止计算梯度

使用`stop_gradient`可以停止计算梯度，示例如下：

In [108]:
from mindspore import ParameterTuple


class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(Tensor(np.array([6], np.float32)), name='w')
        self.b = Parameter(Tensor(np.array([1.0], np.float32)), name='b')

    def construct(self, x):
        out = x * self.w + self.b
        # 停止梯度更新，out对梯度计算无贡献
        out = stop_gradient(out)
        return out


class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)


x = Tensor([6], dtype=mstype.float32)
output = GradNetWrtX(Net())(x)
print(f"wgrad: {output[0]}\nbgrad: {output[1]}")

wgrad: [0.]
bgrad: [0.]


[WARNING] OPTIMIZER(19940,7f1b6134b740,python):2022-02-12-01:41:52.800.577 [mindspore/ccsrc/frontend/optimizer/ad/dfunctor.cc:803] GetPrimalUser] J operation has no relevant primal call in the same graph. Func graph: 1796_construct_wrapper.441, J user: 1796_construct_wrapper.441:construct{[0]: 442, [1]: x, [2]: ValueNode<UMonad> U}
